# The goal of this workbook is to take the sorted data and make it a bit friendlier to work with for what we are doing
The main observation is that as is each game is presented twice (once for each of the two teams playing), in this notebook, we shall take that data and compress in half so each game is only represented once in a single line. Furthermore, we shall add new columns to the csv files that correspond to useful statistics that we need such as the previous runs scored and allowed for each of the teams

In [49]:
#We begin by importing pandas
import pandas as pd

In [50]:
#We shall now read in our files
gl_2010 = pd.read_csv("./sorted_game_logs/gl2010_sorted.csv")

In [51]:
#We shall take a look at the data
gl_2010.head()

,index,Date,team,opponent,runs_scored,runs_allowed,game_number,at_bats,hits,opponent_at_bats,opponent_hits,win_loss,win_frac_actual,win_frac_cummulative_pythagorean,win_frac_cummulative_pythagenpat
0,81,20100405,ANA,MIN,6,3,1,33,9,32,7,1.0,1.000000,0.800000,0.786213
1,82,20100406,ANA,MIN,3,5,2,33,8,32,9,0.0,0.500000,0.558621,0.548292
2,83,20100407,ANA,MIN,2,4,3,35,9,34,7,0.0,0.333333,0.456604,0.468054
3,84,20100408,ANA,MIN,1,10,4,34,8,37,11,0.0,0.250000,0.229299,0.297089
4,85,20100409,ANA,OAK,4,10,5,35,8,41,13,0.0,0.200000,0.200000,0.275311


In [52]:
#First we see that the first column of the index isn't useful, so we shall delete that 
gl_2010 = gl_2010.drop("index", axis = 1)

In [53]:
gl_2010.head()

,Date,team,opponent,runs_scored,runs_allowed,game_number,at_bats,hits,opponent_at_bats,opponent_hits,win_loss,win_frac_actual,win_frac_cummulative_pythagorean,win_frac_cummulative_pythagenpat
0,20100405,ANA,MIN,6,3,1,33,9,32,7,1.0,1.000000,0.800000,0.786213
1,20100406,ANA,MIN,3,5,2,33,8,32,9,0.0,0.500000,0.558621,0.548292
2,20100407,ANA,MIN,2,4,3,35,9,34,7,0.0,0.333333,0.456604,0.468054
3,20100408,ANA,MIN,1,10,4,34,8,37,11,0.0,0.250000,0.229299,0.297089
4,20100409,ANA,OAK,4,10,5,35,8,41,13,0.0,0.200000,0.200000,0.275311


In [54]:
#This is a function that takes in a dataframe
#for an individual team, and returns a series
#which gives the cumulative sum of the runs scored
#for all the games prior to that given one
def make_previous_cum_sum_rs(df):
    cumsumlist = [0]
    for x in df.runs_scored.cumsum():
        cumsumlist.append(x)
    cumsumlist.pop()
    return pd.Series(cumsumlist)
#This is an analogous function for runs allowed
def make_previous_cum_sum_ra(df):
    cumsumlist = [0]
    for x in df.runs_allowed.cumsum():
        cumsumlist.append(x)
    cumsumlist.pop()
    return pd.Series(cumsumlist)    

In [55]:
#Gives the list of teams and alphabetizes it
team_list = gl_2010.team.value_counts().keys()
team_list = sorted(team_list)

In [56]:
#This is a function that takes in a dataframe
#for an individual team, and returns a series
#which gives the cumulative sum of the runs scored
#for all the games prior to that given one
def make_previous_cum_sum_rs(df):
    cumsumlist = [0]
    for x in df.runs_scored.cumsum():
        cumsumlist.append(x)
    cumsumlist.pop()
    return pd.Series(cumsumlist)
#This is an analogous function for runs allowed
def make_previous_cum_sum_ra(df):
    cumsumlist = [0]
    for x in df.runs_allowed.cumsum():
        cumsumlist.append(x)
    cumsumlist.pop()
    return pd.Series(cumsumlist)    

In [57]:
#This function takes in our data and the name of the team and 
#returns a dataframe that is just for that specific team
#it then calls a function that adds the additional data we 
#want from what we have
def make_df_for_team(gamelog, team):
    df = gamelog.loc[gamelog.team == team].reset_index(drop = True) 
    #Note we had to reindex otherwise only the first 
    return make_df_from_team_df(df)
    
#This function takes in the dataframe for a specific team
#It then appends new columns with the additional data that we 
#are interested in which we will be using
def make_df_from_team_df(df):
    df.insert(len(df.T),"past_rs", make_previous_cum_sum_rs(df))
    df.insert(len(df.T),"past_ra", make_previous_cum_sum_ra(df))
    return df

In [58]:
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2010, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)

In [59]:
result

,Date,team,opponent,runs_scored,runs_allowed,game_number,at_bats,hits,opponent_at_bats,opponent_hits,win_loss,win_frac_actual,win_frac_cummulative_pythagorean,win_frac_cummulative_pythagenpat,past_rs,past_ra,past_avg_rs,past_avg_ra
0,20100405,ANA,MIN,6,3,1,33,9,32,7,1.0,1.000000,0.800000,0.786213,0,0,1.000000,1.000000
1,20100406,ANA,MIN,3,5,2,33,8,32,9,0.0,0.500000,0.558621,0.548292,6,3,3.000000,1.500000
2,20100407,ANA,MIN,2,4,3,35,9,34,7,0.0,0.333333,0.456604,0.468054,9,8,3.000000,2.666667
3,20100408,ANA,MIN,1,10,4,34,8,37,11,0.0,0.250000,0.229299,0.297089,11,12,2.750000,3.000000
4,20100409,ANA,OAK,4,10,5,35,8,41,13,0.0,0.200000,0.200000,0.275311,12,22,2.400000,4.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,20100928,WAS,PHI,2,1,158,27,3,35,9,1.0,0.430380,0.444856,0.485334,647,724,4.094937,4.582278
4856,20100929,WAS,PHI,1,7,159,30,3,33,8,0.0,0.427673,0.440874,0.484297,649,725,4.081761,4.559748
4857,20101001,WAS,NYN,1,2,160,33,3,33,4,0.0,0.425000,0.440287,0.484187,650,732,4.062500,4.575000
4858,20101002,WAS,NYN,2,7,161,32,6,33,11,0.0,0.422360,0.437123,0.483369,651,734,4.043478,4.559006


In [149]:
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })

In [150]:
df

,date,team1,team2,team1_rs,team2_rs,team1_ra,team2_ra,team1_gn,team2_gn,team1_at_bats,...,team1_win_frac_cum_pythpat,team2_win_frac_cum_pythpat,team1_past_rs,team2_past_rs,team1_past_ra,team2_past_ra,team1_past_avg_rs,team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra


In [180]:
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)

In [181]:
df

,date,team1,team2,team1_rs,team2_rs,team1_ra,team2_ra,team1_gn,team2_gn,team1_at_bats,...,team1_win_frac_cum_pythpat,team2_win_frac_cum_pythpat,team1_past_rs,team2_past_rs,team1_past_ra,team2_past_ra,team1_past_avg_rs,team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra
0,20100405,ANA,MIN,6,3,3,6,1,1,33,...,0.786213,0.213787,0,0,0,0,1.000000,1.000000,1.000000,1.000000
1,20100406,ANA,MIN,3,5,5,3,2,2,33,...,0.548292,0.451708,6,3,3,6,3.000000,1.500000,1.500000,3.000000
2,20100407,ANA,MIN,2,4,4,2,3,3,35,...,0.468054,0.531946,9,8,8,9,3.000000,2.666667,2.666667,3.000000
3,20100408,ANA,MIN,1,10,10,1,4,4,34,...,0.297089,0.702911,11,12,12,11,2.750000,3.000000,3.000000,2.750000
4,20100409,ANA,OAK,4,10,10,4,5,5,35,...,0.275311,0.652496,12,17,22,13,2.400000,3.400000,4.400000,2.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,20100516,TEX,TOR,2,5,5,2,38,39,32,...,0.508080,0.530686,176,193,166,168,4.631579,4.948718,4.368421,4.307692
2426,20100906,TEX,TOR,2,7,7,2,137,137,37,...,0.516145,0.508526,658,635,581,602,4.802920,4.635036,4.240876,4.394161
2427,20100907,TEX,TOR,5,8,8,5,138,138,36,...,0.515294,0.509094,660,642,588,604,4.782609,4.652174,4.260870,4.376812
2428,20100908,TEX,TOR,8,1,1,8,139,139,40,...,0.516693,0.507472,665,650,596,609,4.784173,4.676259,4.287770,4.381295


In [183]:
df.to_csv("./new_sorted_game_logs/gl2010_new_sorted.csv")

,date,team1,team2,team1_rs,team2_rs,team1_ra,team2_ra,team1_gn,team2_gn,team1_at_bats,...,team1_win_frac_cum_pythpat,team2_win_frac_cum_pythpat,team1_past_rs,team2_past_rs,team1_past_ra,team2_past_ra,team1_past_avg_rs,team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra
0,20100415,ANA,Date 20100415 t...,2,6,6,2,10,9,31,...,0.354486,0.594523,34,46,54,36,3.4,5.111111,5.4,4.0


In [189]:
gl_2011 = pd.read_csv("./sorted_game_logs/gl2011_sorted.csv")
gl_2011 = gl_2011.drop("index", axis = 1)
team_list = gl_2011.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2011, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)


In [190]:
result

,Date,team,opponent,runs_scored,runs_allowed,game_number,at_bats,hits,opponent_at_bats,opponent_hits,win_loss,win_frac_actual,win_frac_cummulative_pythagorean,win_frac_cummulative_pythagenpat,past_rs,past_ra,past_avg_rs,past_avg_ra
0,20110331,ANA,KCA,4,2,1,38,12,33,7,1.0,1.000000,0.800000,0.761186,0,0,1.000000,1.000000
1,20110401,ANA,KCA,1,2,2,31,6,30,7,0.0,0.500000,0.609756,0.575874,4,2,2.000000,1.000000
2,20110402,ANA,KCA,4,5,3,35,11,35,11,0.0,0.333333,0.500000,0.500000,5,4,1.666667,1.333333
3,20110403,ANA,KCA,9,12,4,55,19,51,14,0.0,0.250000,0.423529,0.443291,9,9,2.250000,2.250000
4,20110405,ANA,TBA,5,3,5,34,9,32,5,1.0,0.400000,0.478733,0.485237,18,21,3.600000,4.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4853,20110924,WAS,ATL,4,1,157,28,6,31,6,1.0,0.490446,0.479928,0.494809,606,634,3.859873,4.038217
4854,20110925,WAS,ATL,3,0,158,33,9,29,4,1.0,0.493671,0.482377,0.495456,610,635,3.860759,4.018987
4855,20110926,WAS,FLO,6,4,159,36,10,35,10,1.0,0.496855,0.484106,0.495908,613,635,3.855346,3.993711
4856,20110927,WAS,FLO,2,3,160,30,5,27,5,0.0,0.493750,0.483378,0.495731,619,639,3.868750,3.993750


In [191]:
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)

In [192]:
df

,date,team1,team2,team1_rs,team2_rs,team1_ra,team2_ra,team1_gn,team2_gn,team1_at_bats,...,team1_win_frac_cum_pythpat,team2_win_frac_cum_pythpat,team1_past_rs,team2_past_rs,team1_past_ra,team2_past_ra,team1_past_avg_rs,team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra
0,20110331,ANA,KCA,4,2,2,4,1,1,38,...,0.761186,0.238814,0,0,0,0,1.000000,1.000000,1.000000,1.000000
1,20110401,ANA,KCA,1,2,2,1,2,2,31,...,0.575874,0.424126,4,2,2,4,2.000000,1.000000,1.000000,2.000000
2,20110402,ANA,KCA,4,5,5,4,3,3,35,...,0.500000,0.500000,5,4,4,5,1.666667,1.333333,1.333333,1.666667
3,20110403,ANA,KCA,9,12,12,9,4,4,55,...,0.443291,0.556709,9,9,9,9,2.250000,2.250000,2.250000,2.250000
4,20110405,ANA,TBA,5,3,3,5,5,4,34,...,0.485237,0.210372,18,3,21,12,3.600000,0.750000,4.200000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,20110723,TEX,TOR,5,4,4,5,101,101,35,...,0.527722,0.504536,501,478,418,463,4.960396,4.732673,4.138614,4.584158
2425,20110724,TEX,TOR,0,3,3,0,102,102,31,...,0.526517,0.505465,506,482,422,468,4.960784,4.725490,4.137255,4.588235
2426,20110729,TEX,TOR,2,3,3,2,107,106,36,...,0.527691,0.504897,540,500,448,485,5.046729,4.716981,4.186916,4.575472
2427,20110730,TEX,TOR,3,0,0,3,108,107,34,...,0.528394,0.503949,542,503,451,487,5.018519,4.700935,4.175926,4.551402


In [193]:
df.to_csv("./new_sorted_game_logs/gl2011_new_sorted.csv")

In [194]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2012_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2012_new_sorted.csv")

In [195]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2013_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2013_new_sorted.csv")

In [196]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2014_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2014_new_sorted.csv")

In [197]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2015_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2015_new_sorted.csv")

In [198]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2016_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2016_new_sorted.csv")

In [199]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2017_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2017_new_sorted.csv")

In [200]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2018_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2018_new_sorted.csv")

In [201]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2019_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2019_new_sorted.csv")

In [202]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2020_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2020_new_sorted.csv")

In [203]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2021_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2021_new_sorted.csv")

In [204]:
gl_2012 = pd.read_csv("./sorted_game_logs/gl2022_sorted.csv")
gl_2012 = gl_2012.drop("index", axis = 1)
team_list = gl_2012.team.value_counts().keys()
team_list = sorted(team_list)
#Make a list for the data frame for each team
frames = []
for team in team_list:
    df = make_df_for_team(gl_2012, team)#for each data frame we get the rest of the data we want
    frames.append(df)  #adds the dataframe for that team to the list
result = pd.concat(frames) #puts together all the dataframes into a single datafram
result.insert(len(result.T),"past_avg_rs",(result.past_rs/result.game_number) + (result.game_number == 1)*1)
result.insert(len(result.T),"past_avg_ra",(result.past_ra/result.game_number) + (result.game_number == 1)*1)
result = result.reset_index()
result = result.drop("index", axis = 1)
#Create a list that is the length of the entire thing
df = pd.DataFrame({"date":[], "team1":[], "team2":[], "team1_rs":[], "team2_rs":[], "team1_ra":[], "team2_ra":[], "team1_gn":[], "team2_gn":[], "team1_at_bats":[], "team2_at_bats":[], "team1_hits":[], "team2_hits":[], "team1_win":[], "team2_win":[], "team1_win_frac_acutal":[], "team2_win_frac_actual":[], "team1_win_frac_cum_pyth":[],"team2_win_frac_cum_pyth":[],"team1_win_frac_cum_pythpat":[],"team2_win_frac_cum_pythpat":[], "team1_past_rs":[], "team2_past_rs":[],"team1_past_ra":[],"team2_past_ra":[], "team1_past_avg_rs":[],"team2_past_avg_rs":[],"team1_past_avg_ra":[],"team2_past_avg_ra":[] })
rows = list(range(0,len(result)))
for i in rows:
    team1_index = i
    date = result.iloc[i].Date
    team1 = result.iloc[i].team
    team1_rs = result.iloc[i].runs_scored
    team1_ra = result.iloc[i].runs_allowed
    team1_gn = result.iloc[i].game_number
    team1_at_bats = result.iloc[i].at_bats
    team1_hits = result.iloc[i].hits
    team1_win = result.iloc[i].win_loss
    team1_win_frac_actual = result.iloc[i].win_frac_actual
    team1_win_frac_cum_pyth = result.iloc[i].win_frac_cummulative_pythagorean
    team1_win_frac_cum_pythpat = result.iloc[i].win_frac_cummulative_pythagenpat
    team1_past_rs = result.iloc[i].past_rs
    team1_past_ra = result.iloc[i].past_ra
    team1_past_avg_rs = result.iloc[i].past_avg_rs
    team1_past_avg_ra = result.iloc[i].past_avg_ra
    
    
    team2_index = result.loc[(result.team == result.iloc[i].opponent)
                             & (result.Date == date) 
                             &(result.runs_scored == team1_ra)
                             & (result.runs_allowed == team1_rs)
                             & (result.at_bats == result.iloc[i].opponent_at_bats)].index[0]
    team2 = result.iloc[team2_index].team
    team2_rs = result.iloc[team2_index].runs_scored
    team2_ra = result.iloc[team2_index].runs_allowed
    team2_gn = result.iloc[team2_index].game_number
    team2_at_bats = result.iloc[team2_index].at_bats
    team2_hits = result.iloc[team2_index].hits
    team2_win = result.iloc[team2_index].win_loss
    team2_win_frac_actual = result.iloc[team2_index].win_frac_actual
    team2_win_frac_cum_pyth = result.iloc[team2_index].win_frac_cummulative_pythagorean
    team2_win_frac_cum_pythpat = result.iloc[team2_index].win_frac_cummulative_pythagenpat
    team2_past_rs = result.iloc[team2_index].past_rs
    team2_past_ra = result.iloc[team2_index].past_ra
    team2_past_avg_rs = result.iloc[team2_index].past_avg_rs
    team2_past_avg_ra = result.iloc[team2_index].past_avg_ra
    new_row = pd.DataFrame({"date":[date], "team1":[team1], "team2":[team2], "team1_rs":[team1_rs], "team2_rs":[team2_rs], "team1_ra":[team1_ra], "team2_ra":[team2_ra], "team1_gn":[team1_gn], "team2_gn":[team2_gn], "team1_at_bats":[team1_at_bats], "team2_at_bats":[team2_at_bats], "team1_hits":[team1_hits], "team2_hits":[team2_hits], "team1_win":[team1_win], "team2_win":[team2_win], "team1_win_frac_acutal":[team1_win_frac_acutal], "team2_win_frac_actual":[team2_win_frac_actual], "team1_win_frac_cum_pyth":[team1_win_frac_cum_pyth],"team2_win_frac_cum_pyth":[team2_win_frac_cum_pyth],"team1_win_frac_cum_pythpat":[team1_win_frac_cum_pythpat],"team2_win_frac_cum_pythpat":[team2_win_frac_cum_pythpat], "team1_past_rs":[team1_past_rs], "team2_past_rs":[team2_past_rs],"team1_past_ra":[team1_past_ra],"team2_past_ra":[team2_past_ra], "team1_past_avg_rs":[team1_past_avg_rs],"team2_past_avg_rs":[team2_past_avg_rs],"team1_past_avg_ra":[team1_past_avg_ra],"team2_past_avg_ra":[team2_past_avg_ra] })
    new_row =  [date, team1, team2, team1_rs, team2_rs,
                             team1_ra, team2_ra, team1_gn, team2_gn,
                             team1_at_bats, team2_at_bats, team1_hits,
                             team2_hits, team1_win, team2_win, team1_win_frac_acutal,
                             team2_win_frac_actual, team1_win_frac_cum_pyth,
                             team2_win_frac_cum_pyth,team1_win_frac_cum_pythpat,
                             team2_win_frac_cum_pythpat,team1_past_rs, team2_past_rs,
                             team1_past_ra,team2_past_ra, team1_past_avg_rs,
                             team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra]
    df.loc[len(df)] = new_row
    rows.remove(team2_index)
df.to_csv("./new_sorted_game_logs/gl2022_new_sorted.csv")

In [205]:
df

,date,team1,team2,team1_rs,team2_rs,team1_ra,team2_ra,team1_gn,team2_gn,team1_at_bats,...,team1_win_frac_cum_pythpat,team2_win_frac_cum_pythpat,team1_past_rs,team2_past_rs,team1_past_ra,team2_past_ra,team1_past_avg_rs,team2_past_avg_rs,team1_past_avg_ra,team2_past_avg_ra
0,20220407,ANA,HOU,1,3,3,1,1,1,29,...,0.163086,0.836914,0,0,0,0,1.000000,1.000000,1.000000,1.000000
1,20220408,ANA,HOU,6,13,13,6,2,2,40,...,0.184935,0.815065,1,3,3,1,0.500000,1.500000,1.500000,0.500000
2,20220409,ANA,HOU,2,0,0,2,3,3,27,...,0.295961,0.704039,7,16,16,7,2.333333,5.333333,5.333333,2.333333
3,20220410,ANA,HOU,1,4,4,1,4,4,33,...,0.278869,0.721131,9,16,16,9,2.250000,4.000000,4.000000,2.250000
4,20220411,ANA,MIA,6,2,2,6,5,4,30,...,0.397517,0.377944,10,9,20,10,2.000000,2.250000,4.000000,2.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,20220625,TEX,WAS,3,2,2,3,70,74,29,...,0.502774,0.445660,299,294,295,406,4.271429,3.972973,4.214286,5.486486
2426,20220626,TEX,WAS,4,6,6,4,71,75,33,...,0.501632,0.447573,302,296,297,409,4.253521,3.946667,4.183099,5.453333
2427,20220909,TEX,TOR,3,4,4,3,137,137,34,...,0.497055,0.515075,605,637,617,572,4.416058,4.649635,4.503650,4.175182
2428,20220910,TEX,TOR,7,11,11,7,138,138,39,...,0.496204,0.515758,608,641,621,575,4.405797,4.644928,4.500000,4.166667
